In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings(action = "ignore")

In [2]:
df_song = pd.read_json("./song_meta.json")

In [3]:
df_song_info = df_song[['id' , 'song_name' , 'artist_id_basket', 'artist_name_basket' , 'album_id' , 'album_name' , 'song_gn_gnr_basket' , 'song_gn_dtl_gnr_basket']]

In [4]:
df_song_info['genre_literal'] = df_song_info.song_gn_dtl_gnr_basket.apply(lambda x : (' ').join(x))

In [5]:
df_song_info['artist_literal'] = df_song_info.artist_id_basket.apply(lambda x : str(x[0]))

In [6]:
df_song_info['album_literal'] = df_song_info['album_id'].apply(lambda x : str(x))

In [7]:
df_song_info['all'] = df_song_info['genre_literal'] + " " + df_song_info['artist_literal'] + " " + df_song_info['album_literal']

In [8]:
df_song_info['all']

0                  GN0901 2727 2255639
1           GN1601 GN1606 29966 376431
2                  GN0901 3361 4698747
3         GN1102 GN1101 838543 2644882
4         GN1802 GN1801 560160 2008470
                      ...             
707984             GN2001 166499 65254
707985              GN0901 11837 44141
707986       GN0105 GN0101 437 2662866
707987    GN1807 GN1801 729868 2221722
707988         GN0601 GN0604 895 34663
Name: all, Length: 707989, dtype: object

In [9]:
df_song_info_1 = df_song_info[:50000]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))

song_mat_1 = count_vect.fit_transform(df_song_info_1['all'])

song_sim_1 = cosine_similarity(song_mat_1, song_mat_1)

song_sim_sorted_ind_1 = song_sim_1.argsort()[: , ::-1]

In [11]:
def find_sim_song(song_id, top_n = 10) :
    
    # 인자로 입력된 df_song_info 에서 song_name 추출
    song_id = df_song_info_1[df_song_info_1['id'] == song_id]
    
    # index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    id_index = song_id.index.values
    similar_indexes = song_sim_sorted_ind_1[id_index, :(top_n)]
    
    
    # 추출된 top_n index 출력
    # 데이터프레임에서 index로 사용하기 위해 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df_song_info_1.iloc[similar_indexes]

In [12]:
find_sim_song(1, 10)

[[    1  1078  6688 29430 16884 48111 38892 18394 47288 12319]]


,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,genre_literal,artist_literal,album_literal,all
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,376431,GN1601 GN1606 29966 376431
1078,1078,Schumann : Sonata For Piano No.1 In F- Sharp M...,[29966],[Murray Perahia],48317,"Schumann : Piano Sonata, Op.11 & Kreisleriana,...",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,48317,GN1601 GN1606 29966 48317
6688,6688,"J.S. Bach: French Suite No.1 in D Minor, BWV 8...",[29966],[Murray Perahia],10004046,Johann Sebastian Bach : The French Suites,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,10004046,GN1601 GN1606 29966 10004046
29430,29430,"Chopin : Barcarolle In F-Sharp Major, Op.60",[29966],[Murray Perahia],47306,Chopin : Impromptus,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,47306,GN1601 GN1606 29966 47306
16884,16884,"Bach : English Suite No.6 In D Minor, BWV811 -...",[29966],[Murray Perahia],49708,"Bach : English Suites Nos.1, 3 & 6",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,49708,GN1601 GN1606 29966 49708
48111,48111,Brahms : Handel Variations Op.24 (브람스 : 헨델 변주곡...,[29966],[Murray Perahia],1072776,Brahms : Handel Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,1072776,GN1601 GN1606 29966 1072776
38892,38892,"J.S. Bach: French Suite No.5 In G Major, BWV 8...",[29966],[Murray Perahia],10004046,Johann Sebastian Bach : The French Suites,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,10004046,GN1601 GN1606 29966 10004046
18394,18394,"Bach : Goldberg Variations, BWV988 - Aria",[29966],[Murray Perahia],49719,Bach : Goldberg Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,49719,GN1601 GN1606 29966 49719
47288,47288,Schubert : Piano Quintet In A Major Op.114 D.6...,[828609],[클래식코드],2306626,로맨틱 분위기에 꼭 필요한 클래식 2,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,828609,2306626,GN1601 GN1606 828609 2306626
12319,12319,Trifonov: Rachmaniana - 4. Dolce romantico,[742100],[Daniil Trifonov],2337142,Rachmaninov Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,742100,2337142,GN1601 GN1606 742100 2337142
